In [2]:
%load_ext autoreload
%autoreload 2

import os, json

import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import animation
from datatools.trace_animator import TraceAnimator
from datatools.trace_helper import TraceHelper
from datatools.visualize_helper import VisualizeHelper
from datatools.nba_helper import NBADataHelper, NBADataAnimator
from datatools.nfl_helper import NFLDataHelper

from models import load_model
from models.utils import print_helper, reshape_tensor, get_dataset_config, normalize_tensor

from models.graph_imputer.graph_imputer import BidirectionalGraphImputer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Model evaluating on test data

### Load model

In [ ]:
device = "cuda:0"
trial = 3012
save_path = f"saved/{trial:03d}"

with open(f"{save_path}/params.json", "r") as f:
    params = json.load(f)

if params["model"] == "nrtsi":
    model = load_model(params["model"], params).to(device)

    gap_models_dict = dict()

    gap_models_dict[1] = f"{save_path}/model/nrtsi_state_dict_best_gap_1.pt"
    gap_models_dict[2] = f"{save_path}/model/nrtsi_state_dict_best_gap_2.pt"
    gap_models_dict[4] = f"{save_path}/model/nrtsi_state_dict_best_gap_4.pt"
    gap_models_dict[8] = f"{save_path}/model/nrtsi_state_dict_best_gap_8.pt"
    gap_models_dict[16] = f"{save_path}/model/nrtsi_state_dict_best_gap_16.pt"

    for key in gap_models_dict:
        gap_models_dict[key] = torch.load(gap_models_dict[key], map_location=lambda storage, _: storage)
else:
    state_dict = torch.load(
        f"{save_path}/model/{params['model']}_state_dict_best.pt",
        map_location=lambda storage, _: storage,
    )

    model = load_model(params["model"], params).to(device)
    model.load_state_dict(state_dict)

In [ ]:
model_name = params["model"]
dataset = params["dataset"]

if model_name == "ours":
    physics_loss = params["physics_loss"]
    train_hybrid = params["train_hybrid"]

statistic_metrics = True

print(f"-Model name : {model_name}")
print(f"-Dataset : {dataset}")
print(f"-Compute statistic_metrics : {statistic_metrics}")

In [ ]:
model_keys = ["pred"]
ret_keys = ["n_frames", "n_missings"]
if model.params["model"] == "ours":
    if model.params["physics_loss"]:
        model_keys += ["physics_f", "physics_b"]
    if model.params["train_hybrid"]:
        model_keys += ["static_hybrid", "static_hybrid2", "train_hybrid"]
if statistic_metrics:
    model_keys += ["linear", "knn", "forward"]

    metrics = ["speed", "change_of_step_size", "path_length"]
    ret_keys += [f"{m}_{metric}" for m in model_keys for metric in metrics]

ret_keys += [f"{m}_dist" for m in model_keys]
total_ret = {key: 0 for key in ret_keys}

In [ ]:
metrica_files = ["match1.csv", "match2.csv", "match3_valid.csv", "match3_test.csv"]
metrica_paths = [f"data/metrica_traces/{f}" for f in metrica_files]

nba_files = os.listdir("data/nba_traces")
nba_paths = [f"data/nba_traces/{f}" for f in nba_files]
nba_paths.sort()

nfl_files = os.listdir("data/nfl_traces")
nfl_paths = [f"data/nfl_traces/{f}" for f in nfl_files if f.endswith(".csv")]
nfl_paths.sort()

if dataset == "soccer":
    trace_helper = TraceHelper
    test_data_paths = metrica_paths[3:4]
elif dataset == "basketball":
    trace_helper = NBADataHelper
    test_data_paths = nba_paths[90:]
else: # e.g. "American football"
    trace_helper = NFLDataHelper
    test_data_paths = nfl_paths[0:1]

print(f"Test data paths : {test_data_paths}")

### Run model

In [ ]:
for path in test_data_paths:
    print()
    print(path,":")
    match_traces = pd.read_csv(path, header=0, encoding="utf-8-sig")
    helper = trace_helper(traces=match_traces)

    if model_name == "nrtsi":
        ret, df_dict = helper.predict(model, gap_models=gap_models_dict, statistic_metrics=statistic_metrics, dataset=dataset)
    else:
        ret, df_dict = helper.predict(model, statistic_metrics=statistic_metrics, dataset=dataset)

    for key, value in ret.items():
        total_ret[key] += value
        
    print()

print("Total Performance:")
print_helper(total_ret, model_keys, trial=trial, save_txt=True)

torch.save(helper, f"{save_path}/helper")
torch.save(df_dict, f"{save_path}/df_dict")

## Performance analysis

##### (1) Get Main model results

In [ ]:
trial = 3003
save_path = f"saved/{trial:03d}"
if os.path.isfile(save_path + "/df_dict"):
    helper =  torch.load(save_path + "/helper")
    df_dict = torch.load(save_path + "/df_dict")
    with open(f"{save_path}/params.json", "r") as f:
        params = json.load(f)

##### (2) Add baseline model results

In [ ]:
# trial_dict = {4000 : "brits", 5000 : "naomi", 214 : "nrtsi"} # Metrica
trial_dict = {4003 : "brits", 5001 : "naomi", 6001 : "nrtsi", 9996 : "graphimputer"} # NBA
for (t, model_name) in trial_dict.items():
    save_path = f"saved/{t:03d}"
    if os.path.isfile(save_path + "/df_dict"):
        df_dict_ = torch.load(save_path + "/df_dict")
        df_dict[f"{model_name}_df"] = df_dict_["pred_df"]

In [ ]:
df_dict.keys()

### Animation

##### (1) Soccer Animator

In [ ]:
helper.traces["episode"].unique()

In [ ]:
i0 = 479
i1 = 873

animator = TraceAnimator(
    trace_dict={"main": df_dict["target_df"][i0:i1], "pred": df_dict["train_hybrid_df"][i0:i1]},
    mask = df_dict["mask_df"][i0:i1],
    show_episodes=True,
    show_events=False,
    show_frames=False,
    show_polygon=True,
    annot_cols=None,
)
anim = animator.run()

path = f"animations/trial_{trial}.mp4"

writer = animation.FFMpegWriter(fps=10)
anim.save(path, writer=writer)

##### (2) Basketball Animator

In [ ]:
i0 = 326
i1 = 737
animator = NBADataAnimator(
    trace_dict={"main": df_dict["target_df"][i0:i1], "pred": df_dict["train_hybrid_df"][i0:i1]},
    show_episodes=True,
    show_frames=True,
    masks = df_dict["mask_df"][i0:i1],
)
anim = animator.run()

path = f"animations/trial_{trial}.mp4"

writer = animation.FFMpegWriter(fps=10)
anim.save(path, writer=writer)

### plotting

In [ ]:
plot_mode = "imputed_traj" # "imputed_traj", "dist_heatmap", "weights_heatmap"
dataset = params["dataset"]
visualizer = VisualizeHelper(trial, df_dict, plot_mode, dataset=dataset, helper=helper)
visualizer.valid_episodes()

In [ ]:
visualizer.plot_run(epi_idx=0)
plt.close()